In [4]:
import requests

def get_location_from_ip(amap_api_key):
    """
    调用高德地图 IP 定位 API，获取当前地址
    """
    url = f"https://restapi.amap.com/v3/ip"
    params = {
        'key': amap_api_key  # 高德地图 API 密钥
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == '1':  # 确认请求成功
            # 提取返回的地址信息
            province = data.get('province', '未知省份')
            city = data.get('city', '未知城市')
            adcode = data.get('adcode', '未知行政区代码')
            return {
                'province': province,
                'city': city,
                'adcode': adcode
            }
    return None

# 示例
amap_api_key = "1eac4b127c4bedfe50b5ae79fdc9c12e"  # 替换为你的高德地图 API 密钥
location_info = get_location_from_ip(amap_api_key)

if location_info:
    print(f"当前地址信息: {location_info['province']} {location_info['city']} (行政区代码: {location_info['adcode']})")
else:
    print("无法获取当前地址信息")

当前地址信息: 浙江省 温州市 (行政区代码: 330300)


In [16]:
import requests

def get_location_from_ip(amap_api_key):
    """
    调用高德地图 IP 定位 API，获取大致经纬度和区域信息
    """
    url = "https://restapi.amap.com/v3/ip"
    params = {
        'key': amap_api_key  # 高德地图 API 密钥
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == '1':  # 确认请求成功
            # 提取省、市、矩形范围
            province = data.get('province', '未知省份')
            city = data.get('city', '未知城市')
            rectangle = data.get('rectangle', None)  # 返回矩形区域范围
            return {
                'province': province,
                'city': city,
                'rectangle': rectangle
            }
    return None


def get_detailed_address_from_coordinates(amap_api_key, lng, lat):
    """
    调用高德地图逆地理编码 API，解析经纬度为详细地址
    """
    url = "https://restapi.amap.com/v3/geocode/regeo"
    params = {
        'key': amap_api_key,
        'location': f"{lng},{lat}",  # 经纬度
        'extensions': 'all'  # 返回详细的街道、门牌号、兴趣点等信息
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == '1':  # 请求成功
            regeocode = data.get('regeocode', {})
            formatted_address = regeocode.get('formatted_address', '未知地址')
            address_component = regeocode.get('addressComponent', {})

            # 提取详细的区、街道、门牌号、兴趣点
            district = address_component.get('district', '未知区')
            township = address_component.get('township', '未知街道')
            street = address_component.get('streetNumber', {}).get('street', '未知街道')
            number = address_component.get('streetNumber', {}).get('number', '未知门牌号')
            poi = regeocode.get('pois', [{}])[0].get('name', '无兴趣点')

            return {
                'formatted_address': formatted_address,
                'province': address_component.get('province', '未知省份'),
                'city': address_component.get('city', '未知城市'),
                'district': district,
                'township': township,
                'street': street,
                'number': number,
                'poi': poi
            }
    return None


def get_full_detailed_address(amap_api_key):
    """
    综合 IP 定位和逆地理编码，返回最详细的地址
    """
    # Step 1: 通过 IP 定位获取经纬度范围
    location_info = get_location_from_ip(amap_api_key)
    if not location_info or not location_info.get('rectangle'):
        return "无法获取位置信息"

    # Step 2: 计算矩形范围的中心点作为经纬度
    rectangle = location_info['rectangle']
    lng1, lat1, lng2, lat2 = map(float, rectangle.replace(';', ',').split(','))
    lng = (lng1 + lng2) / 2  # 计算经度中点
    lat = (lat1 + lat2) / 2  # 计算纬度中点

    # Step 3: 调用逆地理编码 API 获取详细地址
    detailed_address = get_detailed_address_from_coordinates(amap_api_key, lng, lat)
    return detailed_address

# 示例
amap_api_key = "1eac4b127c4bedfe50b5ae79fdc9c12e"  # 替换为你的高德地图 API 密钥
address_info = get_full_detailed_address(amap_api_key)

if address_info:
    print(f"详细地址: {address_info['formatted_address']}")
    print(f"省份: {address_info['province']}")
    print(f"城市: {address_info['city']}")
    print(f"区: {address_info['district']}")
    print(f"街道: {address_info['township']}")
    print(f"门牌号: {address_info['street']} {address_info['number']}")
    print(f"兴趣点: {address_info['poi']}")
else:
    print("无法获取详细地址信息")

详细地址: 浙江省温州市鹿城区南汇街道瑞锦组团
省份: 浙江省
城市: 温州市
区: 鹿城区
街道: 南汇街道
门牌号: 划龙桥路 37号
兴趣点: 瑞锦组团


In [20]:
import requests

def get_gps_coordinates():
    """
    模拟 GPS 数据，返回精确的经纬度（实际设备支持时替换为真实 GPS 坐标）
    """
    # 示例坐标，需替换为实际设备获取的 GPS 数据
    lng, lat = 120.699364, 27.994267  # 模拟温州市的具体坐标
    return lng, lat

def get_detailed_address_with_gps(amap_api_key):
    """
    结合 GPS 和逆地理编码 API，获取详细的区、街道和门牌号地址
    """
    # 使用 GPS 获取精确经纬度
    lng, lat = get_gps_coordinates()

    # 调用逆地理编码 API
    url = "https://restapi.amap.com/v3/geocode/regeo"
    params = {
        'key': amap_api_key,
        'location': f"{lng},{lat}",
        'extensions': 'all'  # 确保返回详细地址信息
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == '1':  # 确认请求成功
            regeocode = data.get('regeocode', {})
            formatted_address = regeocode.get('formatted_address', '未知地址')
            address_component = regeocode.get('addressComponent', {})

            # 提取具体地址信息
            province = address_component.get('province', '未知省份')
            city = address_component.get('city', '未知城市')
            district = address_component.get('district', '未知区')
            township = address_component.get('township', '未知街道')
            street = address_component.get('streetNumber', {}).get('street', '未知街道')
            number = address_component.get('streetNumber', {}).get('number', '未知门牌号')
            poi = regeocode.get('pois', [{}])[0].get('name', '无兴趣点')

            return {
                'formatted_address': formatted_address,
                'province': province,
                'city': city,
                'district': district,
                'township': township,
                'street': street,
                'number': number,
                'poi': poi
            }
    return None


# 示例：获取更详细的地址
amap_api_key = "1eac4b127c4bedfe50b5ae79fdc9c12e"  # 替换为你的高德地图 API 密钥
detailed_address = get_detailed_address_with_gps(amap_api_key)

if detailed_address:
    print(f"详细地址: {detailed_address['formatted_address']}")
    print(f"省份: {detailed_address['province']}")
    print(f"城市: {detailed_address['city']}")
    print(f"区: {detailed_address['district']}")
    print(f"街道: {detailed_address['township']}")
    print(f"门牌号: {detailed_address['street']} {detailed_address['number']}")
    print(f"兴趣点: {detailed_address['poi']}")
else:
    print("无法获取详细地址信息")

详细地址: 浙江省温州市鹿城区南汇街道温州市人民政府(北门)
省份: 浙江省
城市: 温州市
区: 鹿城区
街道: 南汇街道
门牌号: 绣山路 321号
兴趣点: 温州市人民政府


In [23]:
import requests
import time

def get_gps_coordinates():
    url = "file:///Users/matsumatsu/Desktop/gps_server.py/gps_server.html"  # 替换为 HTML 文件运行的 URL
    while True:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                gps_data = response.json()
                print(f"实时 GPS 数据: 纬度={gps_data['latitude']}, 经度={gps_data['longitude']}, 精度={gps_data['accuracy']} 米")
            else:
                print("无法获取 GPS 数据")
        except Exception as e:
            print(f"请求失败: {e}")
        time.sleep(5)  # 每 5 秒请求一次

get_gps_coordinates()

请求失败: No connection adapters were found for 'file:///Users/matsumatsu/Desktop/gps_server.py/gps_server.html'
请求失败: No connection adapters were found for 'file:///Users/matsumatsu/Desktop/gps_server.py/gps_server.html'
请求失败: No connection adapters were found for 'file:///Users/matsumatsu/Desktop/gps_server.py/gps_server.html'
请求失败: No connection adapters were found for 'file:///Users/matsumatsu/Desktop/gps_server.py/gps_server.html'
请求失败: No connection adapters were found for 'file:///Users/matsumatsu/Desktop/gps_server.py/gps_server.html'


In [22]:
import asyncio
import websockets

async def gps_server(websocket, path):
    print("客户端已连接")
    try:
        async for message in websocket:
            gps_data = message  # 接收 HTML 页面发送的数据
            print(f"接收到的 GPS 数据: {gps_data}")
    except websockets.exceptions.ConnectionClosed:
        print("客户端已断开连接")

# 启动 WebSocket 服务器
start_server = websockets.serve(gps_server, "localhost", 8080)

asyncio.get_event_loop().run_until_complete(start_server)
asyncio.get_event_loop().run_forever()

RuntimeError: This event loop is already running